<a href="https://colab.research.google.com/github/peterarose/ultrafastultrafast/blob/master/UF2_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installation
You must install UF2 into this instance of Colab in order to run this notebook (this will not install UF2 on your computer).  Install UF2 into this instance of Colab by running the following cell (if you reset the runtime, or close this window and then come back, you will have to evaluate the following cell again)

In [0]:
!pip install ultrafastultrafast

Now import UF2 and other useful pacakges, set up the notebook for ploting

In [0]:
%matplotlib inline
import ultrafastultrafast as uf2
import numpy as np
import matplotlib.pyplot as plt
import yaml
import os

# $\text{UF}^2$
This code is based upon the algorithm Ultrafast Ultrafast ($\text{UF}^2$).  For details see our paper https://doi.org/10.1063/1.5094062.

The fundamental building block of this code is the class uf2.Wavepackets, which implements the $K_j$ and $K_{j^*}$ operators, which are class methods named "up" and "down" respectively, and can be used to calculate the wavepackets associated with any Feynman diagram.  See section Example Diagram below for an example of calculating a single diagram.  See section Signals below for examples of calculating full nonlinear optical signals.

## Requirements 
$\text{UF}^2$ is a method for solving for the perturbative wavepackets of the full Hamiltonian
$$H = H_0 -\boldsymbol{\mu}\cdot \boldsymbol{E}(t)$$
using time-dependent perturbation theory (TDPT). Bold-face symbols are cartesian vectors.  $\text{UF}^2$ requires that the system Hamiltonian, $H_0$ be time-independent, and therefore diagonalizeable.  The eigenvalues, $\hbar\omega_i$ of $H_0$ must be known
$$H_0|i\rangle = \hbar\omega_i|i\rangle.$$
The dipole operator $\boldsymbol{\mu}$ must also be known in the eigenbasis of $H_0$:
$$\boldsymbol{\mu}_{ij} = \langle j|\boldsymbol{\mu}|i\rangle$$

In order to use this code, you must specify the file_path variable to a folder you create

In [0]:
# This is a temporary folder we'll create in Colab to give an example
file_path = '/tmp/dimer_example/'
os.makedirs(file_path,exist_ok=True)

## Unperturbed Hamiltonian H_0
$\text{UF}^2$ should be compatible with any time-independent unperturbed Hamiltonian that can be expressed in some basis as a matrix, and then diagonalized.

In its current implementation, $\text{UF}^2$ relies on the assumption that there are optically separated manifolds, so that $H_0$ is block diagonal with at least two blocks: the ground-state manifold (GSM) and the singly excited manifold (SEM).  There can be more manifolds.  $\text{UF}^2$ is compatible with $n^{th}$-order spectroscopies, the most common of which are $3^{rd}$-order spectroscopies such as Transient Absorption, 2D photon echo, double quantum, and others.  Such spectroscopies are sensitive only to the GSM, SEM and doubly excited manifold (DEM).  Currently, this implementation only handles up to the DEM.

If you have your own eigenvalues, and know the dipole operator in the eigenbasis, you can skip over the Vibronic Eigenstates subsection, and continue reading the necessary files section.  If you would like to use our package for generating and diagonalizing vibronic Hamiltonians, don't skip the Vibronic Eigenstates section!

### Vibronic Eigenstates Package
In order to give an example of $\text{UF}^2$, we include with $\text{UF}^2$ a package in the folder vibronic_eigenstates which has code for generating vibronic Hamiltonians and diagonalizing them.  In order to use this code, you must make a new, empty folder, and place in it a file called simple_params.yaml.  Here's an example of creating that file

In [0]:
with open(file_path+'simple_params.yaml','w+') as f:
  f.write("""
site_energies: [0,0.75]
site_couplings: [0.3307]

vibrations:
  - {'omega_g':1.0, 'displacement':0.2,'site_label':0,'reorganization':0}
  - {'omega_g':1.000001, 'displacement':0.2,'site_label':1,'reorganization':0}

dipoles:
  - [1,0,0]
  - [0,1,0]

num_eigenvalues: 'full'
eigenvalue_precision: 1.0E-1
truncation_size: 5
""")

In [0]:
with open(os.path.join(file_path,'simple_params.yaml')) as yamlstream:
    simple_params = yaml.load(yamlstream)
for key in simple_params.keys():
    print(key,simple_params[key])

To convert this simple parameters file, which is much more user-friendly than the parameters file actually required by the code, we provide a converter:

In [0]:
uf2.vibronic_eigenstates.convert(file_path)

The folder created above should now contain the file params.yaml.  This file can also be opened in a text editor, and is the true parameter file to describe $H_0$.  Advanced users can edit this file directly to gain more control over $H_0$. Note: currently there is no documentation to describe how this works.  Coming soon!

Important: the file params.yaml must exist in the folder in order to generate the necessary eigenvalues and eigenvectors.  The file simple_params.yaml is not sufficient.

The next step is to generate the eigenvalues and eigenvectors of $H_0$.  This is done by calling the following class

In [0]:
uf2.vibronic_eigenstates.EigenGenerator(file_path)

Once the eigenvalues and eigenvectors are obtained, you must express the dipole matrix in the eigenbasis.  Currently we only have the Condon Approximation implemented.  You generate the dipole operator using the following call:

In [0]:
uf2.vibronic_eigenstates.CalculateCartesianDipoleOperator(file_path)

Now the folder file_path should contain a file called mu.npz, which has the dipoles connecting the GSM to the SEM, and the SEM to the DEM

## Necessary files
This section describes the necessary files for $\text{UF}^2$ to work.  If you are using our vibronic_eigenstates package as detailed above in the Vibronic Eigenstates subsection, you don't need to read this (although feel free if you want to see how we store the eigenvalues and dipole operator).

In actuality, $\text{UF}^2$ does not need to know the eigenvectors, so if you have some other way of generating the dipole operator in the eigenbasis, all you need to do is save the following two files in the folder specified by the file_path variable.  You can skip over the above steps, and do not need to use simple_params.yaml or params.yaml.  All that you need is for the folder to conatin the following two numpy archive files:
- eigenvalues.npz - containing up to three keys, each referencing a 1d numpy array
    - 'GSM' - Array of ground state manifold eigenvalues
    - 'SEM' - Array of singly excited manifold eigenvalues
    - 'DEM' (optional) - Array of doubly excited manifold eigenvalues
- mu.npz - containing up to two keys, each referencing a 3d numpy array
    - 'GSM_to_SEM' - Array with indices [i,j,k], representing the dipole operator connecting ground eigenstates (index j) to singly excited eigenstates (index i), with the cartesian coordinate k = 0,1,2 corresponding to k = x,y,z
    - 'SEM_to_DEM' - Array with indices [i,j,k], representing the dipole operator connecting singly excited eigenstates (index j) to doubly excited eigenstates (index i), with the cartesian coordinate k = 0,1,2 corresponding to k = x,y,z
    
See the example_folder included with the github repository for an example of a folder that contains only the necessary files for $\text{UF}^2$, but not the files for using the vibronic_eigenstates package.

In [0]:
eigenvalues_archive = np.load(file_path + 'eigenvalues.npz')
GSM_eigenvalues = eigenvalues_archive['GSM']
SEM_eigenvalues = eigenvalues_archive['SEM']
try:
    DEM_eigenvalues = eigenvalues_archive['DEM']
except KeyError:
    DEM_eigenvalues = 'System does not appear to have a doubly excited manifold'
print('Number of GSM eigenvalues',GSM_eigenvalues.shape)
print('Number of SEM eigenvalues',SEM_eigenvalues.shape)
print('Number of DEM eigenvalues',DEM_eigenvalues.shape)

mu_archive = np.load(file_path + 'mu.npz')
mu_GSM_to_SEM = mu_archive['GSM_to_SEM']
try:
    mu_SEM_to_DEM = mu_archive['SEM_to_DEM']
except KeyError:
    mu_SEM_to_DEM = 'System does not appear to have a doubly excited manifold'

print('Shape of dipole operator connecting GSM to SEM',mu_GSM_to_SEM.shape)
print('Shape of dipole operator connecting SEM to DEM',mu_SEM_to_DEM.shape)

### Dipole Pruning (optional)
You may choose to prune or trim the dipole operator contained in mu.npz down to the fewest necessary states.  This is done by specifying a relative tolerance to use when resolving the dipole operator.  The pruning is implemented as

In [0]:
epsilon_mu = 3.5E-4 # This is a convergence parameter.  We have found this value to be appropriate for this system
dp = uf2.DipolePruning(file_path)
dp.save_boolean_mu(rel_tol = epsilon_mu)

There should now be two new files in the folder we created: mu_pruned.npz and mu_boolean.npz.  mu_pruned.npz is essentially a copy of mu.npz, but with 0's placed in any entries that do not contribute to the given tolerance.  mu_boolean.npz is a boolean array with 2 indices [i,j] that has True for any unaffected values $\mu_{ij}$, and False where ever the pruning process set $\mu_{ij}=0$.

In [0]:
# Let's check!
!ls '/tmp/dimer_example/'

## Example Diagram
Here is an example of how to use the algorithm to calculate the SE contribution of a pump-probe experiment.  We begin by specifying the relevant electric fields. Note that $\text{UF}^2$ does not require analytical pulse shapes.  We use Gaussian pulses here as an example.

In [0]:
def pump(t,sigma):
    """Define pump field"""
    return 1/np.sqrt(2*np.pi)/sigma*np.exp(-t**2/(2*sigma**2))

def probe(t,sigma):
    """Define the probe field"""
    return 1/np.sqrt(2*np.pi)/sigma*np.exp(-t**2/(2*sigma**2))

## Units
Units enter UF2 implicitly via the eigenvalues of the system hamiltonian.  We assume that the eigenvalues are expressed as multiples of some angular frequency $\omega_0$.  All times are expressed in units of $\omega_0^{-1}$, and all frequencies are expressed in units of $\omega_0$.

In [0]:
# Define the pump and probe amplitude functions.  Note that both should be defined centered at t = 0

pump_sigma = 0.8 # in units of omega_0^-1
probe_sigma = 1 # in units of omega_0^-1

# Define the time grid on which the electric fields are to be evaluated
dt = 0.25 # in units of omega_0^-1, time spacing required to resolve all pulse shapes
M = 41 # in units of omega_0^-1, number of points required to resolve all pulse shapes

# Define the time grid on which the wavefunctions will be expressed
total_t = int(200/dt) # in units of omega_0^-1, total number of time points - 
                      # must span from 0 to 5/gamma beyond last delay time point

wp = uf2.Wavepackets(file_path,dt=dt,num_conv_points=M,total_num_time_points=total_t)

# The value of M and dt specified create an array of time points that must be used to evaluate the electric field
# That array is stored inside of the Wavepackets object with attribute efield_t

# pu and pr do not need to be evaluated using an analytical pulse function
pu = pump(wp.efield_t,pump_sigma) # Could be a numpy array of M points describing an arbitrary pulse shape
pr = probe(wp.efield_t,probe_sigma) # Could be a numpy array of M points describing an arbitrary pulse shape

# Pump-probe spectroscopy is calculated to lowest order as four-wave mixing
# where the pump provides the first two interactions, and the probe
# provides the third interaction, and acts as the "fourth interaction" in 
# the form of the local oscillator

wp.efields = [pu,pu,pr,pr]

# The pump is polarized along the 'x' direcetion
# The probe is polarized along the same direction

wp.set_polarization_sequence(['x','x','x','x'])

# Set gamma - effective optical dephasing rate
wp.set_homogeneous_linewidth(0.05)

In [0]:
# Create the wavepackets needed for the SE signal at the specified delay time

delay_time = 10

# Using this delay time, set the interaction times for each of the four interactions
# in the form [pump_time, pump_time, probe_time, probe_time]

wp.pulse_times = [0,0,delay_time,delay_time]

# Construct Perturbative Wavepackets

psi_b = wp.up(wp.psi0,pulse_number = 1)
psi_a = wp.up(wp.psi0,pulse_number = 0)
psi_ac = wp.down(psi_a,pulse_number = 2)

# Calculate the SE contribution to the resulting frequency-resolved spectrum at the specified delay time

# The time-dependent polarization field is the expectation value of the perturbed wavepackets and
# the dipole operator

SE_polarization = wp.dipole_expectation(psi_ac,psi_b)

# Finally convert that polarization into a heterodyne-detected signal

SE_signal = wp.polarization_to_signal(SE_polarization)

# The frequency-detected signal for this value of delay time is plotted here

plt.figure()
plt.plot(wp.w,SE_signal)
plt.xlim([-3,3])
plt.xlabel('Detection Frequency ($\omega_0$)',fontsize=14)
plt.ylabel('Stimulated Emission \n Contribution to TA',fontsize=14)
titletext = plt.title('Delay Time = {}'.format(delay_time) + '$\omega_0^{-1}$',
                     fontsize=16)

## Signals
There are four files located in the signals sub-directory that show examples of how to use uf2 to implement various $n$-wave mixing signals

### Transient Absorption

The file TA_example.py includes a class, TransientAbsorption, that calculates all of the required diagrams for the transient absorption signal using uf2.Wavepackets.  An example of using that class is given here.  To see how all of the diagrams are coded and combined into one signal, please see the code.

In [0]:
pump_sigma = 0.2
probe_sigma = 0.2
dt = min(pump_sigma,probe_sigma)/2
M = 21
total_t = int(250/dt)

In [0]:
# Specify initial state in the GSM. Usually this is the 0 state (the lowest energy eigenstate), but if you are
# interested including thermal averaging, you may want to run several simulations using different starting states
psi0_state = 0

# Initialize object with filepath to saved eigenvalues and dipole operator
ta = uf2.signals.TransientAbsorption(file_path,dt=dt,num_conv_points=M,total_num_time_points=total_t,
                                    initial_state=psi0_state)

# As before, signal fields and polarization sequence must be set

pu = pump(ta.efield_t,pump_sigma) #Using above Gaussian field definition
pr = probe(ta.efield_t,probe_sigma) #Using above Gaussian field definition

# The class TransientAbsorption has a method for setting the four interaction fields given the pump shape and the
# probe shape

ta.set_pulse_shapes(pu,pr)

# The pump is polarized along the 'x' direcetion
# The probe is polarized along the same direcetion

pol_seq = ['x','x','x','x']

ta.set_polarization_sequence(pol_seq)

In [0]:
# Pick delay times of interest.  The code is written in such a way that negative delay times cannot be calculated. 
# This is a limitation of the implementation, not of the method
delay_times = np.arange(0,90,1) #delay times must coincide with time mesh calculation is performed on

# The class TransientAbsorption includes all necessary diagrams for calculating the TA signal

spectra = ta.calculate_pump_probe_spectra_vs_delay_time(delay_times)
ta.save_pump_probe_spectra_vs_delay_time()
ta.plot_pump_probe_spectra(frequency_range = [-5,5],subtract_DC=True,save_fig=True)

In [0]:
ta_iso = uf2.signals.TransientAbsorptionIsotropicAverage(file_path,pol_seq,dt=dt,num_conv_points=M,
                                                         initial_state=psi0_state, total_num_time_points=total_t)
ta_iso.set_pulse_shapes(pu,pr)

In [0]:
spectra_iso = ta_iso.calculate_spectra(delay_times)
ta_iso.save_pump_probe_spectra_vs_delay_time()
ta_iso.plot_pump_probe_spectra(frequency_range = [-5,5],subtract_DC=True,save_fig=True)